In [5]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import pandas as pd
import tensorflow as tf
import os
from tensorflow.keras.callbacks import ModelCheckpoint
import numpy as np

#  Import and read the charity_data.csv.
import pandas as pd 
application_df = pd.read_csv("charity_data.csv")

In [6]:
# Drop the non-beneficial ID columns, 'EIN' and 'NAME'.
application_df.drop(columns=['EIN','NAME'],inplace=True)

In [7]:
# Determine which values to replace if counts are less than ...?
replace_application = [app[1]['APPLICATION_TYPE'] for app in application_df[['APPLICATION_TYPE']].value_counts().reset_index().iterrows() if app[1][0] < 500]

# Replace in dataframe
for app in replace_application:
    application_df.APPLICATION_TYPE = application_df.APPLICATION_TYPE.replace(app,"Other")

In [8]:
# Determine which values to replace if counts are less than ..?
replace_class = [cls[1]['CLASSIFICATION'] for cls in application_df[['CLASSIFICATION']].value_counts().reset_index().iterrows() if cls[1][0] < 1000]

# Replace in dataframe
for cls in replace_class:
    application_df.CLASSIFICATION = application_df.CLASSIFICATION.replace(cls,"Other")

In [9]:
# Generate our categorical variable lists
application_cat = application_df.dtypes[application_df.dtypes == "object"].index.tolist()

In [10]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(application_df[application_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(application_cat)

In [11]:
# Merge one-hot encoded features and drop the originals
application_df = application_df.merge(encode_df, left_index=True, right_index=True).drop(columns=application_cat)

In [12]:
# Split our preprocessed data into our features and target arrays
y = application_df['IS_SUCCESSFUL'].values
X = application_df.drop(columns='IS_SUCCESSFUL').values

# Split the preprocessed data into a training and testing dataset
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

In [13]:
# Create a StandardScaler instances
scaler = StandardScaler()

# Fit the StandardScaler
X_scaler = scaler.fit(X_train)

# Scale the data
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [14]:
# Create a method that creates a new Sequential model with hyperparameter options
def create_model(hp):
    nn_model = tf.keras.models.Sequential()

    # Allow kerastuner to decide which activation function to use in hidden layers
    # activation = hp.Choice('activation',['relu','tanh','sigmoid'])
    
    # Allow kerastuner to decide number of neurons in first layer
    nn_model.add(tf.keras.layers.Dense(units=hp.Int('first_units',
        min_value=32, max_value=512, step=32), activation=hp.Choice('activation',['relu','tanh','sigmoid'])))

    # Allow kerastuner to decide number of hidden layers and neurons in hidden layers
    for i in range(hp.Int('num_layers', 1, 3)):
        nn_model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
            min_value=32, max_value=512, step=32),
            activation=hp.Choice('activation',['relu','tanh','sigmoid'])))
    
    nn_model.add(tf.keras.layers.Dense(units=1, activation="sigmoid"))

    # Compile the model
    nn_model.compile(loss="binary_crossentropy", optimizer='adam', metrics=["accuracy"])
    
    return nn_model

In [15]:
# Import the kerastuner library
import kerastuner as kt

tuner = kt.Hyperband(
    create_model,
    objective="val_accuracy",
    max_epochs=100,
    hyperband_iterations=2,
    directory='app_tuner',
    overwrite=False,
    project_name='app_tuner')

INFO:tensorflow:Reloading Tuner from app_tuner\app_tuner\tuner0.json


In [16]:
# Run the kerastuner search for best hyperparameters
tuner.search(X_train_scaled,y_train,validation_data=(X_test_scaled,y_test))


Search: Running Trial #236

Value             |Best Value So Far |Hyperparameter
384               |96                |first_units
relu              |relu              |activation
1                 |1                 |num_layers
32                |480               |units_0
160               |480               |units_1
160               |320               |units_2
100               |4                 |tuner/epochs
34                |2                 |tuner/initial_epoch
2                 |4                 |tuner/bracket
2                 |1                 |tuner/round
0231              |0037              |tuner/trial_id

Epoch 35/100
804/804 [==============================] - 2s 2ms/step - loss: 0.5415 - accuracy: 0.7367 - val_loss: 0.5545 - val_accuracy: 0.7328
Epoch 36/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5405 - accuracy: 0.7366 - val_loss: 0.5565 - val_accuracy: 0.7313
Epoch 37/100
804/804 [==============================] - 1s 1ms/step - loss: 0.5

KeyboardInterrupt: 

In [17]:
# Get best model hyperparameters
best_hyper = tuner.get_best_hyperparameters(1)[0]
best_hyper.values

{'first_units': 96,
 'activation': 'relu',
 'num_layers': 1,
 'units_0': 480,
 'units_1': 480,
 'units_2': 320,
 'tuner/epochs': 4,
 'tuner/initial_epoch': 2,
 'tuner/bracket': 4,
 'tuner/round': 1,
 'tuner/trial_id': '0037'}

In [18]:
# Evaluate best model against full test data
best_model = tuner.get_best_models(1)[0]
model_loss, model_accuracy = best_model.evaluate(X_test_scaled,y_test,verbose=2)
print(f"Loss: {model_loss}, Accuracy: {model_accuracy}")

268/268 - 0s - loss: 0.5545 - accuracy: 0.7341 - 319ms/epoch - 1ms/step
Loss: 0.5544995665550232, Accuracy: 0.7341107726097107


In [19]:
best_model.save("train_application_optimization.h5")